In [3]:
import sys
sys.path.insert(0, '..')

import polars as pl
from pathlib import Path

# Load preprocessed data
data_file = Path('../data/processed/df_sample_split_preprocessed.parquet')
df = pl.read_parquet(data_file)

print(f"Preprocessed data loaded!")
print(f"Shape: {df.shape}")
print(f"\nColumn names: {df.columns}")
print(f"\nData types:")
print(df.schema)
print(f"\nFirst few rows:")
print(df.head())

Preprocessed data loaded!
Shape: (588776, 23)

Column names: ['id', 'session', 'electoralTerm', 'firstName', 'lastName', 'politicianId', 'speechContent', 'factionId', 'documentUrl', 'positionShort', 'positionLong', 'date', 'speech_length', 'paragraph_number', 'paragraph_length', 'tokens', 'token_count', 'tokens_no_stopwords', 'token_count_no_stopwords', 'tokens_clean', 'token_count_clean', 'tokens_lemma', 'token_count_lemma']

Data types:
Schema([('id', Int64), ('session', Int64), ('electoralTerm', Int64), ('firstName', String), ('lastName', String), ('politicianId', Int64), ('speechContent', String), ('factionId', Int64), ('documentUrl', String), ('positionShort', String), ('positionLong', String), ('date', String), ('speech_length', Int64), ('paragraph_number', Int64), ('paragraph_length', Int64), ('tokens', List(String)), ('token_count', UInt32), ('tokens_no_stopwords', List(String)), ('token_count_no_stopwords', UInt32), ('tokens_clean', List(String)), ('token_count_clean', UInt32)

In [4]:
# Join tokens back into full strings using native Polars (much faster than map_elements)
import polars as pl

# Use native Polars list.join() instead of slow map_elements
df = df.with_columns(
    pl.col('tokens_lemma').list.join(' ').alias('text_lemmatized')
)

# Also create versions from other token types for comparison
df = df.with_columns(
    pl.col('tokens_clean').list.join(' ').alias('text_clean')
)

df = df.with_columns(
    pl.col('tokens_no_stopwords').list.join(' ').alias('text_no_stopwords')
)

print("Tokens joined into full strings!")
print(f"\nDataframe shape: {df.shape}")
print(f"New columns: {df.columns}")

print(f"\nSample text from lemmatized tokens (first paragraph):")
print(f"  {df['text_lemmatized'][0]}")

Tokens joined into full strings!

Dataframe shape: (588776, 26)
New columns: ['id', 'session', 'electoralTerm', 'firstName', 'lastName', 'politicianId', 'speechContent', 'factionId', 'documentUrl', 'positionShort', 'positionLong', 'date', 'speech_length', 'paragraph_number', 'paragraph_length', 'tokens', 'token_count', 'tokens_no_stopwords', 'token_count_no_stopwords', 'tokens_clean', 'token_count_clean', 'tokens_lemma', 'token_count_lemma', 'text_lemmatized', 'text_clean', 'text_no_stopwords']

Sample text from lemmatized tokens (first paragraph):
  geehrt Herr Präsident Friedrich Kollegin Kollege letzter Satz Kollege Hunko begrüßen ausdrücklich anschließen mal Gescheit linker Ecke begeistern


In [6]:
# TF-IDF Vectorization of lemmatized texts
from sklearn.feature_extraction.text import TfidfVectorizer
from tqdm import tqdm
import numpy as np

print("TF-IDF Vectorization")
print("Converting lemmatized texts to TF-IDF vectors...")

# Initialize TfidfVectorizer with German-specific parameters
tfidf_vectorizer = TfidfVectorizer(
    max_features=1000,          # Limit vocabulary to top 1000 features
    min_df=2,                   # Minimum document frequency
    max_df=0.8,                 # Maximum document frequency (80% of docs)
    ngram_range=(1, 2),         # Use unigrams and bigrams
    sublinear_tf=True,          # Apply sublinear term frequency scaling
    norm='l2'                   # L2 normalization
)

# Convert lemmatized texts to TF-IDF vectors
print("Processing documents...")
tfidf_matrix = tfidf_vectorizer.fit_transform(tqdm(df['text_lemmatized'], desc="Vectorizing", unit="doc"))

print(f"\nTF-IDF Vectorization complete!")
print(f"TF-IDF matrix shape: {tfidf_matrix.shape}")
print(f"  Samples (documents): {tfidf_matrix.shape[0]}")
print(f"  Features (vocabulary): {tfidf_matrix.shape[1]}")
print(f"  Sparsity: {(1 - tfidf_matrix.nnz / (tfidf_matrix.shape[0] * tfidf_matrix.shape[1])) * 100:.2f}%")

# Get feature names (vocabulary)
feature_names = np.array(tfidf_vectorizer.get_feature_names_out())
print(f"\nVocabulary size: {len(feature_names)}")
print(f"Sample features: {feature_names[:20]}")

# Convert sparse matrix to dense for inspection
tfidf_dense = tfidf_matrix.toarray()

# Show top TF-IDF terms for first document
print(f"\nTop 10 TF-IDF terms for first document:")
top_indices = tfidf_dense[0].argsort()[-10:][::-1]
for idx in top_indices:
    print(f"  {feature_names[idx]}: {tfidf_dense[0, idx]:.4f}")

# Store TF-IDF matrix and vectorizer for later use
print(f"\nTF-IDF vectorizer and matrix ready for topic modeling!")


TF-IDF Vectorization
Converting lemmatized texts to TF-IDF vectors...
Processing documents...


Vectorizing: 100%|██████████| 588776/588776 [00:22<00:00, 25804.58doc/s]



TF-IDF Vectorization complete!
TF-IDF matrix shape: (588776, 1000)
  Samples (documents): 588776
  Features (vocabulary): 1000
  Sparsity: 99.35%

Vocabulary size: 1000
Sample features: ['abgeordneter' 'abkommen' 'ablehnen' 'abschaffen' 'abschließend'
 'abschluss' 'absolut' 'afd' 'afghanistan' 'afrika' 'aktiv' 'aktuell'
 'akzeptieren' 'all' 'alleine' 'alt' 'amt' 'anbieten' 'anderer'
 'anerkennung']

Top 10 TF-IDF terms for first document:
  begrüßen: 0.3556
  satz: 0.3313
  geehrt herr: 0.3256
  kollege: 0.3217
  ausdrücklich: 0.3020
  mal: 0.2799
  geehrt: 0.2726
  herr präsident: 0.2610
  präsident: 0.2518
  letzter: 0.2338

TF-IDF vectorizer and matrix ready for topic modeling!


In [7]:
# LDA Topic Modeling with OPTIMIZED parameters for better coherence
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import CountVectorizer
from tqdm import tqdm
import numpy as np
import time

print("LDA Topic Modeling (FULL DATASET)")
print("="*80)
start_time = time.time()

# Use FULL dataset
SAMPLE_FRACTION = 1.0  # Use 100% of data
if SAMPLE_FRACTION < 1.0:
    df_lda = df.sample(fraction=SAMPLE_FRACTION, seed=42)
    print(f"Using {SAMPLE_FRACTION*100:.0f}% sample: {df_lda.shape[0]:,} documents")
else:
    df_lda = df
    print(f"Using full dataset: {df_lda.shape[0]:,} documents")

# German stopwords - viel wichtiger als englische!
german_stopwords = [
    'der', 'die', 'das', 'den', 'dem', 'des', 'ein', 'eine', 'einer', 'einem', 'einen', 'eines',
    'und', 'oder', 'aber', 'doch', 'wenn', 'weil', 'dass', 'ob', 'als', 'wie', 'so', 'auch',
    'noch', 'schon', 'nur', 'sehr', 'mehr', 'viel', 'andere', 'anderen', 'anderem', 'anderer',
    'ist', 'sind', 'war', 'waren', 'wird', 'werden', 'wurde', 'wurden', 'hat', 'haben', 'hatte', 'hatten',
    'kann', 'können', 'konnte', 'konnten', 'muss', 'müssen', 'musste', 'mussten', 'soll', 'sollen',
    'will', 'wollen', 'wollte', 'wollten', 'darf', 'dürfen', 'mag', 'mögen', 'möchte', 'möchten',
    'ich', 'du', 'er', 'sie', 'es', 'wir', 'ihr', 'sie', 'Sie', 'mein', 'dein', 'sein', 'ihr', 'unser', 'euer',
    'mich', 'dich', 'sich', 'uns', 'euch', 'mir', 'dir', 'ihm', 'ihr', 'ihnen', 'Ihnen',
    'dieser', 'diese', 'dieses', 'diesem', 'diesen', 'jener', 'jene', 'jenes', 'jenem', 'jenen',
    'welcher', 'welche', 'welches', 'welchem', 'welchen', 'was', 'wer', 'wen', 'wem', 'wessen',
    'hier', 'dort', 'wo', 'wohin', 'woher', 'wann', 'warum', 'weshalb', 'wieso', 'nun', 'jetzt', 'dann', 'damals',
    'immer', 'nie', 'niemals', 'manchmal', 'oft', 'selten', 'bereits', 'bisher', 'künftig',
    'über', 'unter', 'vor', 'nach', 'zwischen', 'neben', 'bei', 'mit', 'ohne', 'für', 'gegen', 'durch',
    'von', 'zu', 'bis', 'seit', 'während', 'wegen', 'trotz', 'statt', 'anstatt', 'außer', 'innerhalb', 'außerhalb',
    'herr', 'frau', 'kollege', 'kollegin', 'kollegen', 'damen', 'herren', 'präsident', 'präsidentin',
    'abgeordnete', 'abgeordneter', 'abgeordneten', 'minister', 'ministerin', 'bundesregierung',
    'antrag', 'gesetzentwurf', 'drucksache', 'fraktion', 'ausschuss', 'debatte', 'rede',
    'damit', 'dazu', 'davon', 'dafür', 'dagegen', 'darauf', 'darin', 'daraus', 'darum', 'dabei',
    'also', 'allerdings', 'jedoch', 'dennoch', 'trotzdem', 'deshalb', 'deswegen', 'daher',
    'eigentlich', 'natürlich', 'selbstverständlich', 'tatsächlich', 'wirklich', 'ganz', 'gar',
    'etwa', 'ungefähr', 'circa', 'fast', 'beinahe', 'kaum', 'eben', 'gerade', 'halt', 'mal',
    'ja', 'nein', 'nicht', 'kein', 'keine', 'keiner', 'keinem', 'keinen', 'nichts', 'niemand',
    'etwas', 'alles', 'alle', 'allem', 'allen', 'aller', 'jeder', 'jede', 'jedes', 'jedem', 'jeden',
    'viele', 'vielen', 'vielem', 'vieler', 'einige', 'einigen', 'einigem', 'einiger', 'mehrere', 'mehreren',
    'beide', 'beiden', 'beider', 'solche', 'solchen', 'solchem', 'solcher', 'welch', 'manch', 'manche',
    'Jahr', 'Jahren', 'Prozent', 'Million', 'Millionen', 'Milliarde', 'Milliarden', 'Euro', 'Mark',
    'heute', 'gestern', 'morgen', 'Jahr', 'Jahre', 'Monat', 'Monate', 'Woche', 'Wochen', 'Tag', 'Tage',
    'erste', 'ersten', 'erstem', 'erster', 'zweite', 'zweiten', 'zweitem', 'zweiter', 'dritte', 'dritten',
    'neue', 'neuen', 'neuem', 'neuer', 'neues', 'alte', 'alten', 'altem', 'alter', 'altes',
    'große', 'großen', 'großem', 'großer', 'großes', 'kleine', 'kleinen', 'kleinem', 'kleiner', 'kleines',
    'gut', 'guten', 'gutem', 'guter', 'gutes', 'schlecht', 'schlechten', 'schlechtem', 'schlechter',
    'wichtig', 'wichtigen', 'wichtigem', 'wichtiger', 'wichtiges', 'richtig', 'richtigen', 'richtigem',
    'möglich', 'möglichen', 'möglichem', 'möglicher', 'mögliches', 'nötig', 'nötigen', 'nötigem',
    'sagen', 'gesagt', 'sage', 'sagst', 'sagt', 'sagten', 'meinen', 'gemeint', 'meine', 'meinst', 'meint',
    'wissen', 'gewusst', 'weiß', 'weißt', 'wisst', 'wusste', 'glauben', 'geglaubt', 'glaube', 'glaubst',
    'denken', 'gedacht', 'denke', 'denkst', 'denkt', 'dachte', 'finden', 'gefunden', 'finde', 'findest',
    'kommen', 'gekommen', 'komme', 'kommst', 'kommt', 'kam', 'kamen', 'gehen', 'gegangen', 'gehe', 'gehst',
    'machen', 'gemacht', 'mache', 'machst', 'macht', 'machte', 'machten', 'tun', 'getan', 'tue', 'tust',
    'geben', 'gegeben', 'gebe', 'gibst', 'gibt', 'gab', 'gaben', 'nehmen', 'genommen', 'nehme', 'nimmst',
    'lassen', 'gelassen', 'lasse', 'lässt', 'ließ', 'ließen', 'stehen', 'gestanden', 'stehe', 'stehst',
    'bringen', 'gebracht', 'bringe', 'bringst', 'bringt', 'brachte', 'brachten',
    'land', 'länder', 'deutschland', 'deutschen', 'deutscher', 'deutsche', 'bundesrepublik', 'bund', 'bundes'
]

# OPTIMIZED CountVectorizer für bessere Topics
count_vectorizer = CountVectorizer(
    max_features=5000,       # Mehr Features für bessere Unterscheidung
    min_df=20,               # Höher: nur Wörter die in mind. 20 Docs vorkommen
    max_df=0.7,              # Niedriger: Wörter die in >70% vorkommen raus
    ngram_range=(1, 1),      # Nur Unigrams - Bigrams verwässern oft Topics
    stop_words=german_stopwords,
    token_pattern=r'\b[a-zäöüß]{3,}\b'  # Nur Wörter mit min 3 Buchstaben
)

# STEP 1: Vectorization
print(f"\n[STEP 1/3] Vectorizing {df_lda.shape[0]:,} documents...")
vectorize_start = time.time()
doc_term_matrix = count_vectorizer.fit_transform(tqdm(df_lda['text_lemmatized'], desc="Vectorizing"))
vectorize_time = time.time() - vectorize_start

print(f"✓ Vectorization complete in {vectorize_time:.1f}s")
print(f"  Document-Term Matrix Shape: {doc_term_matrix.shape}")
print(f"  Sparsity: {(1 - doc_term_matrix.nnz / (doc_term_matrix.shape[0] * doc_term_matrix.shape[1])) * 100:.2f}%")

# STEP 2: LDA Training
n_topics = 15
print(f"\n[STEP 2/3] Training LDA model with {n_topics} topics...")
print(f"  This may take 10-20 minutes for {df_lda.shape[0]:,} documents...")

lda_model = LatentDirichletAllocation(
    n_components=n_topics,
    random_state=42,
    max_iter=15,             # Reduced iterations for faster training
    learning_method='online', # Online learning with progress
    learning_offset=50.,
    batch_size=4096,         # Larger batches for faster processing
    doc_topic_prior=0.1,     # Alpha
    topic_word_prior=0.01,   # Beta
    n_jobs=-1,
    verbose=1                # Shows iteration progress
)

train_start = time.time()
lda_model.fit(doc_term_matrix)
train_time = time.time() - train_start

print(f"\n✓ LDA Training complete in {train_time/60:.1f} minutes")
print(f"  Perplexity: {lda_model.perplexity(doc_term_matrix):.4f}")

# STEP 3: Extract and display topics
print(f"\n[STEP 3/3] Extracting topic terms...")
feature_names = np.array(count_vectorizer.get_feature_names_out())

print(f"\n" + "="*80)
print("TOP 15 TERMS FOR EACH TOPIC")
print("="*80)

n_top_words = 15
for topic_idx, topic in enumerate(lda_model.components_):
    top_words_idx = topic.argsort()[-n_top_words:][::-1]
    top_words = [feature_names[i] for i in top_words_idx]
    print(f"\nTopic {topic_idx + 1:2d}: {', '.join(top_words)}")

total_time = time.time() - start_time
print(f"\n" + "="*80)
print(f"TOTAL TIME: {total_time/60:.1f} minutes")
print("="*80)

LDA Topic Modeling (FULL DATASET)
Using full dataset: 588,776 documents

[STEP 1/3] Vectorizing 588,776 documents...


Vectorizing: 100%|██████████| 588776/588776 [00:08<00:00, 68263.80it/s]


✓ Vectorization complete in 10.1s
  Document-Term Matrix Shape: (588776, 5000)
  Sparsity: 99.84%

[STEP 2/3] Training LDA model with 15 topics...
  This may take 10-20 minutes for 588,776 documents...
iteration: 1 of max_iter: 15
iteration: 2 of max_iter: 15
iteration: 3 of max_iter: 15
iteration: 4 of max_iter: 15
iteration: 5 of max_iter: 15
iteration: 6 of max_iter: 15
iteration: 7 of max_iter: 15
iteration: 8 of max_iter: 15
iteration: 9 of max_iter: 15
iteration: 10 of max_iter: 15
iteration: 11 of max_iter: 15
iteration: 12 of max_iter: 15
iteration: 13 of max_iter: 15
iteration: 14 of max_iter: 15
iteration: 15 of max_iter: 15

✓ LDA Training complete in 11.2 minutes
  Perplexity: 2506.7026

[STEP 3/3] Extracting topic terms...

TOP 15 TERMS FOR EACH TOPIC

Topic  1: thema, punkt, letzter, diskussion, diskutieren, sprechen, seite, woche, hören, rolle, führen, aktuell, stunde, bericht, spielen

Topic  2: unternehmen, brauchen, arbeitsplatz, wirtschaft, schaffen, mensch, entwickl

In [5]:
%pip install gensim

Note: you may need to restart the kernel to use updated packages.


In [9]:
# Coherence Score Berechnung (Memory-Efficient)
from sklearn.metrics.pairwise import cosine_similarity
from tqdm import tqdm
import numpy as np
import time

print("="*80)
print("MODEL QUALITY EVALUATION")
print("="*80)

# Calculate Perplexity
print("\n[1/3] Calculating Perplexity...")
perplexity = lda_model.perplexity(doc_term_matrix)
print(f"✓ Perplexity Score: {perplexity:.4f}")

# Calculate log-likelihood
print("\n[2/3] Calculating Log-Likelihood...")
log_likelihood = lda_model.score(doc_term_matrix)
log_likelihood_per_doc = log_likelihood / doc_term_matrix.shape[0]
print(f"✓ Total Log-Likelihood: {log_likelihood:.4f}")
print(f"✓ Log-Likelihood per document: {log_likelihood_per_doc:.4f}")

# Memory-efficient coherence calculation using sparse operations
print("\n[3/3] Calculating Topic Coherence (memory-efficient)...")

def calculate_topic_coherence_sparse(lda_model, doc_term_matrix, feature_names, top_n=10):
    """
    Calculate coherence using sparse matrix operations (memory-efficient)
    Uses PMI-like measure based on co-document frequency
    """
    from scipy.sparse import csc_matrix
    
    # Keep matrix sparse - convert to CSC for efficient column slicing
    dtm_sparse = csc_matrix(doc_term_matrix)
    n_docs = dtm_sparse.shape[0]
    
    coherence_scores = []
    
    for topic_idx, topic in enumerate(tqdm(lda_model.components_, desc="Topics")):
        # Get top word indices for this topic
        top_word_indices = topic.argsort()[-top_n:][::-1]
        
        # Calculate co-occurrence coherence using document frequency
        topic_coherence = 0
        pair_count = 0
        
        for i in range(len(top_word_indices)):
            for j in range(i + 1, len(top_word_indices)):
                w1_idx = top_word_indices[i]
                w2_idx = top_word_indices[j]
                
                # Get document frequencies (sparse column operations)
                w1_docs = (dtm_sparse[:, w1_idx].toarray().flatten() > 0)
                w2_docs = (dtm_sparse[:, w2_idx].toarray().flatten() > 0)
                
                # Co-occurrence: documents where both words appear
                co_occur = np.sum(w1_docs & w2_docs)
                w1_freq = np.sum(w1_docs)
                
                # PMI-like score: log(P(w1,w2) / P(w1))
                if co_occur > 0 and w1_freq > 0:
                    coherence = np.log((co_occur * n_docs) / (w1_freq * np.sum(w2_docs)) + 1e-10)
                    topic_coherence += coherence
                pair_count += 1
        
        avg_coherence = topic_coherence / pair_count if pair_count > 0 else 0
        coherence_scores.append(avg_coherence)
    
    return np.mean(coherence_scores), coherence_scores

# Get feature names
feature_names = np.array(count_vectorizer.get_feature_names_out())

# Calculate coherence
avg_coherence, topic_coherences = calculate_topic_coherence_sparse(lda_model, doc_term_matrix, feature_names)

print(f"\n" + "="*80)
print("COHERENCE SCORES BY TOPIC")
print("="*80)
for topic_idx, coherence in enumerate(topic_coherences):
    print(f"  Topic {topic_idx + 1:2d}: {coherence:.4f}")

print(f"\n" + "="*80)
print("MODEL QUALITY SUMMARY")
print("="*80)
print(f"\nDataset Statistics:")
print(f"  Number of Documents: {doc_term_matrix.shape[0]:,}")
print(f"  Vocabulary Size: {doc_term_matrix.shape[1]:,}")
print(f"  Number of Topics: {n_topics}")

print(f"\nModel Metrics:")
print(f"  Perplexity: {perplexity:.2f}")
print(f"  Average Topic Coherence (PMI): {avg_coherence:.4f}")
print(f"  Log-Likelihood: {log_likelihood:.2f}")
print(f"  Log-Likelihood per Doc: {log_likelihood_per_doc:.4f}")

print(f"\n" + "="*80)
print("INTERPRETATION")
print("="*80)
print(f"""
📊 PERPLEXITY: {perplexity:.2f}
   - Measures how well the model predicts held-out data
   - Lower is better (typical range: 100-5000 for LDA)
   - Your score of {perplexity:.0f} is {'moderate' if perplexity < 3000 else 'high'} for this dataset size
   
📊 TOPIC COHERENCE (PMI): {avg_coherence:.4f}
   - Measures how semantically related top words are within each topic
   - Higher is better (range typically -2 to +2 for PMI)
   - Positive values indicate words co-occur more than expected by chance
   - Your score of {avg_coherence:.2f} indicates {'good' if avg_coherence > 0 else 'weak'} topic coherence
   
📊 LOG-LIKELIHOOD: {log_likelihood_per_doc:.4f} per document
   - Measures how well the model fits the training data
   - Higher (less negative) is better
   
🔍 OVERALL ASSESSMENT:
   {"✓ Model shows reasonable topic structure" if avg_coherence > 0 else "⚠ Topics may need refinement"}
   {"✓ Perplexity is within acceptable range" if perplexity < 4000 else "⚠ High perplexity - consider more iterations or different parameters"}
   
💡 RECOMMENDATION:
   For parliamentary speech analysis, consider using the ParlBERT classifier
   (next section) for more interpretable, predefined political topic categories.
""")

print("="*80)

MODEL QUALITY EVALUATION

[1/3] Calculating Perplexity...
✓ Perplexity Score: 2506.7026

[2/3] Calculating Log-Likelihood...
✓ Total Log-Likelihood: -39438937.7703
✓ Log-Likelihood per document: -66.9846

[3/3] Calculating Topic Coherence (memory-efficient)...


Topics: 100%|██████████| 15/15 [00:12<00:00,  1.16it/s]


COHERENCE SCORES BY TOPIC
  Topic  1: 0.9722
  Topic  2: 1.3431
  Topic  3: 0.6975
  Topic  4: 0.9873
  Topic  5: 1.0988
  Topic  6: 0.9904
  Topic  7: 1.1567
  Topic  8: 1.1532
  Topic  9: 1.7897
  Topic 10: 1.2901
  Topic 11: 1.0457
  Topic 12: 1.1227
  Topic 13: 0.9069
  Topic 14: 1.6222
  Topic 15: 0.4741

MODEL QUALITY SUMMARY

Dataset Statistics:
  Number of Documents: 588,776
  Vocabulary Size: 5,000
  Number of Topics: 15

Model Metrics:
  Perplexity: 2506.70
  Average Topic Coherence (PMI): 1.1100
  Log-Likelihood: -39438937.77
  Log-Likelihood per Doc: -66.9846

INTERPRETATION

📊 PERPLEXITY: 2506.70
   - Measures how well the model predicts held-out data
   - Lower is better (typical range: 100-5000 for LDA)
   - Your score of 2507 is moderate for this dataset size

📊 TOPIC COHERENCE (PMI): 1.1100
   - Measures how semantically related top words are within each topic
   - Higher is better (range typically -2 to +2 for PMI)
   - Positive values indicate words co-occur more th

In [10]:
# Save LDA Topic Model Results to DataFrames using Polars (with all metadata)
import polars as pl
from pathlib import Path
from tqdm import tqdm
import json
import numpy as np

print("="*80)
print("SAVING LDA TOPIC MODEL RESULTS")
print("="*80)

# 1. Get topic assignments for each document
print("\n1. Extracting topic assignments for each document...")
doc_topic_dist = lda_model.transform(doc_term_matrix)  # Shape: (n_docs, n_topics)

# Get dominant topic for each document
dominant_topics = doc_topic_dist.argmax(axis=1)
dominant_topic_prob = doc_topic_dist.max(axis=1)

# Get all metadata columns from original dataframe
metadata_cols = ['id', 'session', 'electoralTerm', 'firstName', 'lastName', 'politicianId', 
                 'factionId', 'documentUrl', 'positionShort', 'positionLong', 'date', 
                 'speech_length', 'paragraph_number', 'paragraph_length', 'speechContent']
df_metadata = df.select(metadata_cols)

# Create Polars dataframe with document-level topic assignments and all metadata
df_lda_topics_docs = df_metadata.with_columns(
    pl.Series('dominant_topic', dominant_topics.tolist()),
    pl.Series('dominant_topic_prob', dominant_topic_prob.tolist())
)

print(f"Document-topic assignments shape: {df_lda_topics_docs.shape}")
print(f"Columns: {df_lda_topics_docs.columns}")
print(f"\nFirst 10 rows:")
print(df_lda_topics_docs.head(10))

# 2. Get top terms for each topic
print(f"\n2. Extracting top terms for each topic...")
n_top_words = 15
topics_terms_list = []

for topic_idx, topic_components in enumerate(tqdm(lda_model.components_, desc="Extracting topic terms")):
    # Get indices of top words
    top_words_idx = topic_components.argsort()[-n_top_words:][::-1]
    
    # Get words and their weights
    top_words = feature_names[top_words_idx]
    top_weights = topic_components[top_words_idx]
    
    topics_terms_list.append({
        'topic': topic_idx,
        'top_terms': ', '.join(top_words),
        'top_terms_list': json.dumps(top_words.tolist()),
        'weights': json.dumps(top_weights.tolist())
    })

df_lda_topics_terms = pl.DataFrame(topics_terms_list)

print(f"Topics-terms dataframe shape: {df_lda_topics_terms.shape}")
print(f"\nFirst 5 topics:")
print(df_lda_topics_terms.select(['topic', 'top_terms']).head())

# 3. Create topic quality metrics dataframe
print(f"\n3. Creating topic quality metrics...")
df_lda_topic_metrics = pl.DataFrame({
    'topic': list(range(n_topics)),
    'coherence_score': topic_coherences
})

print(f"Topic metrics shape: {df_lda_topic_metrics.shape}")
print(f"\nTopic metrics:")
print(df_lda_topic_metrics.head())

# 4. Save all LDA results to parquet and csv
print(f"\n4. Saving results to disk...")
output_dir = Path('../data/processed')
output_dir.mkdir(parents=True, exist_ok=True)

# Save document-topic assignments with all metadata
parquet_path_docs = output_dir / 'topic_document_assignments_lda.parquet'
csv_path_docs = output_dir / 'topic_document_assignments_lda.csv'

df_lda_topics_docs.write_parquet(parquet_path_docs)
df_lda_topics_docs.write_csv(csv_path_docs)

print(f"\n✓ LDA Document-topic assignments saved (with all metadata):")
print(f"  Parquet: {parquet_path_docs}")
print(f"  CSV: {csv_path_docs}")
print(f"  Total columns: {df_lda_topics_docs.width}")

# Save topic terms
parquet_path_terms = output_dir / 'topic_terms_lda.parquet'
csv_path_terms = output_dir / 'topic_terms_lda.csv'

df_lda_topics_terms.write_parquet(parquet_path_terms)
df_lda_topics_terms.select(['topic', 'top_terms']).write_csv(csv_path_terms)

print(f"\n✓ LDA Topic terms saved:")
print(f"  Parquet: {parquet_path_terms}")
print(f"  CSV: {csv_path_terms}")

# Save topic metrics
parquet_path_metrics = output_dir / 'topic_metrics_lda.parquet'
csv_path_metrics = output_dir / 'topic_metrics_lda.csv'

df_lda_topic_metrics.write_parquet(parquet_path_metrics)
df_lda_topic_metrics.write_csv(csv_path_metrics)

print(f"\n✓ LDA Topic metrics saved:")
print(f"  Parquet: {parquet_path_metrics}")


SAVING LDA TOPIC MODEL RESULTS

1. Extracting topic assignments for each document...
Document-topic assignments shape: (588776, 17)
Columns: ['id', 'session', 'electoralTerm', 'firstName', 'lastName', 'politicianId', 'factionId', 'documentUrl', 'positionShort', 'positionLong', 'date', 'speech_length', 'paragraph_number', 'paragraph_length', 'speechContent', 'dominant_topic', 'dominant_topic_prob']

First 10 rows:
shape: (10, 17)
┌─────────┬─────────┬────────────┬───────────┬───┬────────────┬────────────┬───────────┬───────────┐
│ id      ┆ session ┆ electoralT ┆ firstName ┆ … ┆ paragraph_ ┆ speechCont ┆ dominant_ ┆ dominant_ │
│ ---     ┆ ---     ┆ erm        ┆ ---       ┆   ┆ length     ┆ ent        ┆ topic     ┆ topic_pro │
│ i64     ┆ i64     ┆ ---        ┆ str       ┆   ┆ ---        ┆ ---        ┆ ---       ┆ b         │
│         ┆         ┆ i64        ┆           ┆   ┆ i64        ┆ str        ┆ i64       ┆ ---       │
│         ┆         ┆            ┆           ┆   ┆            

Extracting topic terms: 100%|██████████| 15/15 [00:00<00:00, 5995.86it/s]

Topics-terms dataframe shape: (15, 4)

First 5 topics:
shape: (5, 2)
┌───────┬─────────────────────────────────┐
│ topic ┆ top_terms                       │
│ ---   ┆ ---                             │
│ i64   ┆ str                             │
╞═══════╪═════════════════════════════════╡
│ 0     ┆ thema, punkt, letzter, diskuss… │
│ 1     ┆ unternehmen, brauchen, arbeits… │
│ 2     ┆ herzlich, haus, programm, sich… │
│ 3     ┆ schritt, kommune, ort, leisten… │
│ 4     ┆ gesetz, bürger, regelung, mögl… │
└───────┴─────────────────────────────────┘

3. Creating topic quality metrics...
Topic metrics shape: (15, 2)

Topic metrics:
shape: (5, 2)
┌───────┬─────────────────┐
│ topic ┆ coherence_score │
│ ---   ┆ ---             │
│ i64   ┆ f64             │
╞═══════╪═════════════════╡
│ 0     ┆ 0.97222         │
│ 1     ┆ 1.343107        │
│ 2     ┆ 0.697478        │
│ 3     ┆ 0.98727         │
│ 4     ┆ 1.098763        │
└───────┴─────────────────┘

4. Saving results to disk...



✓ LDA Document-topic assignments saved (with all metadata):
  Parquet: ..\data\processed\topic_document_assignments_lda.parquet
  CSV: ..\data\processed\topic_document_assignments_lda.csv
  Total columns: 17

✓ LDA Topic terms saved:
  Parquet: ..\data\processed\topic_terms_lda.parquet
  CSV: ..\data\processed\topic_terms_lda.csv

✓ LDA Topic metrics saved:
  Parquet: ..\data\processed\topic_metrics_lda.parquet


# Alternative Topic Classification: ParlBERT-Topic-German - not applied because of time ineffectiveness

The LDA topic modeling results above showed limited coherence and interpretability. The topics were not well-defined and difficult to assign meaningful labels.

As an alternative, we use the **ParlBERT-Topic-German** model from HuggingFace ([chkla/parlbert-topic-german](https://huggingface.co/chkla/parlbert-topic-german)), which is a fine-tuned BERT model specifically trained for German parliamentary speech topic classification. This model classifies text into predefined political topic categories, providing more interpretable and consistent results for parliamentary speeches.

In [8]:
# Install transformers if not already installed
%pip install transformers torch

Note: you may need to restart the kernel to use updated packages.


In [1]:
# ParlBERT-Topic-German Classification (ONNX OPTIMIZED for CPU)
# ONNX Runtime is 2-3x faster than PyTorch on CPU!
%pip install optimum[onnxruntime] -q

from optimum.onnxruntime import ORTModelForSequenceClassification
from transformers import AutoTokenizer
import polars as pl
from pathlib import Path
import numpy as np

print("="*80)
print("PARLBERT-TOPIC-GERMAN CLASSIFICATION (ONNX OPTIMIZED)")
print("="*80)

# Load model with ONNX Runtime (much faster on CPU!)
print("\nLoading ParlBERT-Topic-German model with ONNX optimization...")
model_name = "chkla/parlbert-topic-german"

tokenizer = AutoTokenizer.from_pretrained(model_name)
# This will automatically convert to ONNX and cache it
model = ORTModelForSequenceClassification.from_pretrained(
    model_name, 
    export=True,  # Convert to ONNX
    provider="CPUExecutionProvider"
)

print("✓ ONNX Model loaded successfully!")

# Get label mapping
id2label = model.config.id2label
print(f"\nAvailable topics: {list(id2label.values())}")

# Test the model
test_text = "Das Sachgebiet Investive Ausgaben des Bundes Bundesfinanzminister Apel hat gemäß BMF Finanznachrichten vom 1. Januar erklärt, die Investitionsquote des Bundes sei in den letzten zehn Jahren nahezu konstant geblieben."
inputs = tokenizer(test_text, return_tensors="pt", truncation=True, max_length=512)
outputs = model(**inputs)
probs = outputs.logits.softmax(dim=-1).numpy()
pred_id = probs.argmax()
confidence = probs.max()

print(f"\nTest classification:")
print(f"  Text: {test_text[:80]}...")
print(f"  Predicted topic: {id2label[pred_id]}")
print(f"  Confidence: {confidence:.4f}")


Note: you may need to restart the kernel to use updated packages.


c:\Users\olive\miniconda3\envs\parl_speech\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


PARLBERT-TOPIC-GERMAN CLASSIFICATION (ONNX OPTIMIZED)

Loading ParlBERT-Topic-German model with ONNX optimization...


c:\Users\olive\miniconda3\envs\parl_speech\Lib\site-packages\transformers\modeling_attn_mask_utils.py:196: TracerWarning: torch.tensor results are registered as constants in the trace. You can safely ignore this warning if you use this function to create tensors out of constant variables that would be the same every time you call this function. In any other case, this might cause the trace to be incorrect.
  inverted_mask = torch.tensor(1.0, dtype=dtype) - expanded_mask


✓ ONNX Model loaded successfully!

Available topics: ['Macroeconomics', 'Civil', 'Health', 'Agriculture', 'Labor', 'Education', 'Environment', 'Energy', 'Immigration', 'Transportation', 'Law', 'Social', 'Housing', 'Domestic', 'Defense', 'Technology', 'Foreign', 'International', 'Government', 'Public', 'Culture']

Test classification:
  Text: Das Sachgebiet Investive Ausgaben des Bundes Bundesfinanzminister Apel hat gemäß...
  Predicted topic: Macroeconomics
  Confidence: 0.9967


In [2]:
# Classify documents using ParlBERT (MEMORY-SAFE with Sampling)
from tqdm import tqdm
import numpy as np
import gc

# Check if df is loaded, if not load it
if 'df' not in dir():
    print("Loading data first...")
    import polars as pl
    from pathlib import Path
    data_file = Path('../data/processed/df_sample_split_preprocessed.parquet')
    df = pl.read_parquet(data_file)
    print(f"Data loaded: {df.shape[0]:,} documents")

print("\n" + "="*80)
print("PARLBERT CLASSIFICATION (with Stratified Sampling)")
print("="*80)

# OPTION: Use sampling for faster processing
# Set to 1.0 for full dataset, or lower (e.g., 0.1 = 10%) for faster testing
SAMPLE_FRACTION = 0.1  # Process 10% of data (~59k documents)

if SAMPLE_FRACTION < 1.0:
    # Stratified sample by factionId to maintain party distribution
    df_classify = df.sample(fraction=SAMPLE_FRACTION, seed=42)
    print(f"Using {SAMPLE_FRACTION*100:.0f}% sample: {df_classify.shape[0]:,} documents")
else:
    df_classify = df
    print(f"Processing full dataset: {df_classify.shape[0]:,} documents")

# Get texts
texts = df_classify['speechContent'].to_list()

# Small batch size to avoid memory issues
batch_size = 32
total_batches = (len(texts) + batch_size - 1) // batch_size
print(f"Batch size: {batch_size}")
print(f"Total batches: {total_batches}")
print(f"Estimated time: ~{total_batches * 2 / 60:.1f} minutes")

# Results
topic_labels = []
topic_scores = []

# Process in batches with memory management
pbar = tqdm(range(0, len(texts), batch_size), desc="Classifying", unit="batch", total=total_batches)

for i in pbar:
    batch_texts = [t if t else "" for t in texts[i:i+batch_size]]
    
    # Tokenize
    inputs = tokenizer(
        batch_texts,
        padding=True,
        truncation=True,
        max_length=128,  # Reduced further for memory
        return_tensors="pt"
    )
    
    # Inference
    outputs = model(**inputs)
    probs = outputs.logits.softmax(dim=-1).numpy()
    
    # Get predictions
    pred_ids = probs.argmax(axis=-1)
    confidences = probs.max(axis=-1)
    
    # Store results
    topic_labels.extend([id2label[pid] for pid in pred_ids])
    topic_scores.extend(confidences.tolist())
    
    # Update progress
    pbar.set_postfix({'docs': f'{len(topic_labels):,}/{len(texts):,}'})
    
    # Clear memory every 100 batches
    if i > 0 and i % 3200 == 0:
        gc.collect()

# Cleanup
del texts
gc.collect()

print(f"\n✓ Classification complete!")
print(f"  Documents classified: {len(topic_labels):,}")
print(f"  Average confidence: {sum(topic_scores)/len(topic_scores):.4f}")

# Store the sampled dataframe for later saving
df_classified = df_classify


Loading data first...
Data loaded: 588,776 documents

PARLBERT CLASSIFICATION (with Stratified Sampling)
Using 10% sample: 58,877 documents
Batch size: 32
Total batches: 1840
Estimated time: ~61.3 minutes


Classifying:   5%|▌         | 95/1840 [10:53<3:20:00,  6.88s/batch, docs=3,040/58,877]


KeyboardInterrupt: 

In [ ]:
# Analyze topic distribution
print("\n" + "="*80)
print("TOPIC DISTRIBUTION ANALYSIS")
print("="*80)

# Get unique topics and their counts
topic_counts = pl.DataFrame({'topic_label': topic_labels}).group_by('topic_label').len().sort('len', descending=True)

print("\nTopic distribution:")
for row in topic_counts.iter_rows(named=True):
    percentage = (row['len'] / len(topic_labels)) * 100
    print(f"  {row['topic_label']:30s}: {row['len']:5d} documents ({percentage:5.2f}%)")

print(f"\nTotal unique topics: {topic_counts.shape[0]}")


TOPIC DISTRIBUTION ANALYSIS

Topic distribution:
  Government                    :   180 documents (43.69%)
  Civil                         :    86 documents (20.87%)
  International                 :    22 documents ( 5.34%)
  Law                           :    19 documents ( 4.61%)
  Macroeconomics                :    19 documents ( 4.61%)
  Labor                         :    16 documents ( 3.88%)
  Social                        :    15 documents ( 3.64%)
  Defense                       :    11 documents ( 2.67%)
  Agriculture                   :    10 documents ( 2.43%)
  Domestic                      :     8 documents ( 1.94%)
  Environment                   :     7 documents ( 1.70%)
  Technology                    :     5 documents ( 1.21%)
  Housing                       :     4 documents ( 0.97%)
  Health                        :     4 documents ( 0.97%)
  Education                     :     2 documents ( 0.49%)
  Energy                        :     2 documents ( 0.49%)
  Fore

In [ ]:
# Save ParlBERT Classification Results to DataFrames
print("\n" + "="*80)
print("SAVING PARLBERT TOPIC CLASSIFICATION RESULTS")
print("="*80)

# Get all metadata columns from the CLASSIFIED dataframe (may be sampled)
metadata_cols = ['id', 'session', 'electoralTerm', 'firstName', 'lastName', 'politicianId', 
                 'factionId', 'documentUrl', 'positionShort', 'positionLong', 'date', 
                 'speech_length', 'paragraph_number', 'paragraph_length', 'speechContent']
df_metadata = df_classified.select(metadata_cols)

# Create numeric topic IDs for compatibility with existing analysis
unique_topics = sorted(list(set(topic_labels)))
topic_to_id = {topic: idx for idx, topic in enumerate(unique_topics)}
topic_ids = [topic_to_id[label] for label in topic_labels]

# Create Polars dataframe with document-level topic assignments and all metadata
df_topics_docs = df_metadata.with_columns(
    pl.Series('dominant_topic', topic_ids),
    pl.Series('dominant_topic_prob', topic_scores),
    pl.Series('topic_label', topic_labels)
)

print(f"\nDocument-topic assignments shape: {df_topics_docs.shape}")
print(f"Columns: {df_topics_docs.columns}")
print(f"\nFirst 10 rows (selected columns):")
print(df_topics_docs.select(['id', 'firstName', 'lastName', 'dominant_topic', 'topic_label', 'dominant_topic_prob']).head(10))

# Create topic terms/labels dataframe (mapping topic IDs to their labels)
df_topics_terms = pl.DataFrame({
    'topic': list(range(len(unique_topics))),
    'top_terms': unique_topics,  # For ParlBERT, the "top_terms" is the topic label itself
    'topic_label': unique_topics
})

print(f"\nTopics terms/labels dataframe shape: {df_topics_terms.shape}")
print(f"\nTopic mapping:")
print(df_topics_terms)

# Create topic metrics dataframe (confidence scores per topic)
df_topic_stats = df_topics_docs.group_by('dominant_topic').agg([
    pl.col('dominant_topic_prob').mean().alias('mean_confidence'),
    pl.col('dominant_topic_prob').std().alias('std_confidence'),
    pl.len().alias('document_count')
]).sort('dominant_topic')

# Join with topic labels
df_topic_metrics = df_topic_stats.join(
    df_topics_terms.select(['topic', 'topic_label']),
    left_on='dominant_topic',
    right_on='topic',
    how='left'
).select(['dominant_topic', 'topic_label', 'mean_confidence', 'std_confidence', 'document_count'])
df_topic_metrics = df_topic_metrics.rename({'dominant_topic': 'topic'})

print(f"\nTopic metrics shape: {df_topic_metrics.shape}")
print(f"\nTopic metrics:")
print(df_topic_metrics)



SAVING PARLBERT TOPIC CLASSIFICATION RESULTS

Document-topic assignments shape: (412, 18)
Columns: ['id', 'session', 'electoralTerm', 'firstName', 'lastName', 'politicianId', 'factionId', 'documentUrl', 'positionShort', 'positionLong', 'date', 'speech_length', 'paragraph_number', 'paragraph_length', 'speechContent', 'dominant_topic', 'dominant_topic_prob', 'topic_label']

First 10 rows (selected columns):
shape: (10, 6)
┌────────┬───────────┬──────────┬────────────────┬─────────────┬─────────────────────┐
│ id     ┆ firstName ┆ lastName ┆ dominant_topic ┆ topic_label ┆ dominant_topic_prob │
│ ---    ┆ ---       ┆ ---      ┆ ---            ┆ ---         ┆ ---                 │
│ i64    ┆ str       ┆ str      ┆ i64            ┆ str         ┆ f64                 │
╞════════╪═══════════╪══════════╪════════════════╪═════════════╪═════════════════════╡
│ 738998 ┆ burkhard  ┆ lischka  ┆ 8              ┆ Government  ┆ 0.942619            │
│ 738998 ┆ burkhard  ┆ lischka  ┆ 8              ┆ Go

In [ ]:
# Save all BERT results to parquet and csv
output_dir = Path('../data/processed')
output_dir.mkdir(parents=True, exist_ok=True)

# Save document-topic assignments with all metadata (main result for merging)
parquet_path_docs = output_dir / 'topic_document_assignments_bert.parquet'
csv_path_docs = output_dir / 'topic_document_assignments_bert.csv'

df_topics_docs.write_parquet(parquet_path_docs)
df_topics_docs.write_csv(csv_path_docs)

print(f"\n✓ BERT Document-topic assignments saved (with all metadata):")
print(f"  Parquet: {parquet_path_docs}")
print(f"  CSV: {csv_path_docs}")
print(f"  Total columns: {df_topics_docs.width}")

# Save topic terms/labels
parquet_path_terms = output_dir / 'topic_terms_bert.parquet'
csv_path_terms = output_dir / 'topic_terms_bert.csv'

df_topics_terms.write_parquet(parquet_path_terms)
df_topics_terms.write_csv(csv_path_terms)

print(f"\n✓ BERT Topic terms/labels saved:")
print(f"  Parquet: {parquet_path_terms}")
print(f"  CSV: {csv_path_terms}")

# Save topic metrics
parquet_path_metrics = output_dir / 'topic_metrics_bert.parquet'
csv_path_metrics = output_dir / 'topic_metrics_bert.csv'

df_topic_metrics.write_parquet(parquet_path_metrics)
df_topic_metrics.write_csv(csv_path_metrics)

print(f"\n✓ BERT Topic metrics saved:")
print(f"  Parquet: {parquet_path_metrics}")
print(f"  CSV: {csv_path_metrics}")

print(f"\n" + "="*80)
print("PARLBERT TOPIC CLASSIFICATION RESULTS SAVED SUCCESSFULLY!")
print("="*80)


✓ BERT Document-topic assignments saved (with all metadata):
  Parquet: ..\data\processed\topic_document_assignments_bert.parquet
  CSV: ..\data\processed\topic_document_assignments_bert.csv
  Total columns: 18

✓ BERT Topic terms/labels saved:
  Parquet: ..\data\processed\topic_terms_bert.parquet
  CSV: ..\data\processed\topic_terms_bert.csv

✓ BERT Topic metrics saved:
  Parquet: ..\data\processed\topic_metrics_bert.parquet
  CSV: ..\data\processed\topic_metrics_bert.csv

PARLBERT TOPIC CLASSIFICATION RESULTS SAVED SUCCESSFULLY!
